In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling
from tqdm import tqdm
from transformers import GPT2Tokenizer

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/home/gerold/projects/gpt2-all-in/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
tokenizer.

In [17]:
dataset = load_dataset('stas/openwebtext-10k', split='train')

Reusing dataset openwebtext10k (/home/gerold/.cache/huggingface/datasets/stas___openwebtext10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b)


In [25]:
def encode(example):
    return tokenizer(
        example['text'], truncation=True, padding='max_length',
        max_length=1024,
        return_tensors='np', return_attention_mask=False)

In [26]:
encode(dataset[0])

{'input_ids': array([[   32,  7093, 10327, ..., 12407,   286,   564]])}

In [27]:
dataset = dataset.map(
    encode, batched=True, remove_columns=['text'])

100%|██████████| 10/10 [00:44<00:00,  4.48s/ba]


In [30]:
dataset.set_format(type='torch', columns=['input_ids'])

In [31]:
dataset[0]

{'input_ids': tensor([   32,  7093, 10327,  ..., 12407,   286,   564])}

In [32]:
loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [33]:
next(iter(loader))

{'input_ids': tensor([[ 3633,  2692,   802,  ..., 16052,   379,   281],
         [22846,  2969,  3535,  ..., 50256, 50256, 50256],
         [16977,    25, 28634,  ..., 50256, 50256, 50256],
         ...,
         [ 1870,    11,  3737,  ..., 50256, 50256, 50256],
         [  464, 14672, 36880,  ...,   198, 10603,  4421],
         [  464,  3452,  5726,  ..., 50256, 50256, 50256]])}

In [2]:
dataset = load_dataset('Skylion007/openwebtext', streaming=True, split='train')

In [3]:
dataset = dataset.with_format("torch")

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/home/gerold/projects/gpt2-all-in/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
def encode(example):
    return tokenizer(
        example['text'], truncation=True, padding='max_length', max_length=1024,
        return_tensors='pt', return_attention_mask=False)

In [6]:
dataset = dataset.map(encode, batched=True, remove_columns=['text'])

In [13]:
seed, buffer_size = 10, 10_000
dataset = dataset.shuffle(seed, buffer_size)

In [7]:
out = next(iter(dataset))

In [8]:
assert isinstance(dataset, torch.utils.data.IterableDataset)


In [9]:
loader = DataLoader(dataset, shuffle=False, batch_size=8)

In [10]:
for batch in loader:
    print(batch)
    break

{'input_ids': tensor([[13924,    12,   559,  ...,   284,  3236,  3146],
        [14282,  7705,   286,  ...,   198, 26747,  2714],
        [  464,  9317,  6241,  ...,   262,  5940,  2056],
        ...,
        [   32, 18192, 34678,  ..., 50256, 50256, 50256],
        [12906,  2773,  7312,  ..., 50256, 50256, 50256],
        [ 8888,    11, 20182,  ..., 50256, 50256, 50256]])}


In [13]:
from gpt2_ai.config import GPT2Config
from gpt2_ai.model import GPT2

In [36]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%autoreload 2

In [46]:
conf = GPT2Config(d_model=768, n_head=12, n_layer=12)
model = GPT2(conf)
model.count_params()

Number of trainable parameters: 123,703,633


In [47]:
1024 * 768

786432

In [48]:
768 * 56000

43008000

In [49]:
conf = GPT2Config(d_model=1024, n_head=16, n_layer=24)

In [50]:
model = GPT2(conf)
model.count_params()

Number of trainable parameters: 303,468,625


In [43]:
12 * 1024 ** 2 * 24

301989888

In [26]:
batch.keys()

NameError: name 'batch' is not defined

In [28]:
loader = DataLoader(dataset, shuffle=False, batch_size=8, collate_fn=encode)

In [29]:
next(iter(loader))

TypeError: object of type 'IterableDataset' has no len()

In [ ]:
DataCollatorForLanguageModeling()

In [26]:
for ex in loader:
    print(ex)
    break

TypeError: object of type 'IterableDataset' has no len()

In [25]:
ex['input_ids'].shape

torch.Size([1024])

In [7]:
ex['input_ids'].shape

torch.Size([1024])

In [10]:
len(ex['text'])

5516

In [12]:
len(tokenizer.decode(ex['input_ids']))

4726

In [ ]:
ex['text']

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also